## 지역 경제 환경 변수 정리

In [1]:
from google.colab import drive
drive.mount('/content/drive')

drive_path = '/content/drive/MyDrive/new_project1/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import re

import geopandas as gpd


# 제출용 코드에서 깔끔하게 보이게,..
import warnings
warnings.filterwarnings("ignore")

In [3]:
finance = pd.read_csv('/content/drive/MyDrive/new_project1/data/경제/금융기관_20240411161531.csv', header = 1)
finance.head()

,동별(1),동별(2),동별(3),금융기관
0,합계,소계,소계,1734
1,합계,종로구,소계,103
2,합계,종로구,사직동,21
3,합계,종로구,삼청동,3
4,합계,종로구,부암동,1


In [4]:
finance.columns = ['합계', '시군구', '읍면동', '금융기관']
finance.drop(columns = '합계', inplace =True)
finance = finance[finance['읍면동'] != '소계']
finance.reset_index(drop = True, inplace = True)
finance.head()

,시군구,읍면동,금융기관
0,종로구,사직동,21
1,종로구,삼청동,3
2,종로구,부암동,1
3,종로구,평창동,2
4,종로구,무악동,1


## 동 이름에서  < . · >제거
- 예시 - 종로1.2.3.4가동

In [5]:
finance['읍면동'] = finance['읍면동'].apply(lambda x: re.sub(r'\·', '', x))
finance['읍면동'] = finance['읍면동'].apply(lambda x: re.sub(r'\.', '', x))
finance[finance['읍면동'].str.contains('\.') | finance['읍면동'].str.contains('\·') ]


,시군구,읍면동,금융기관


In [6]:
finance['구동'] = finance['시군구'] + ' ' +finance['읍면동']
finance.head()

,시군구,읍면동,금융기관,구동
0,종로구,사직동,21,종로구 사직동
1,종로구,삼청동,3,종로구 삼청동
2,종로구,부암동,1,종로구 부암동
3,종로구,평창동,2,종로구 평창동
4,종로구,무악동,1,종로구 무악동


In [7]:
# 금융기관이 없는 것은 - 로 값이 입력되어있다.
# 이를 0으로 변환
finance['금융기관'] = finance['금융기관'].apply(lambda x: 0 if x =='-' else x)
finance['금융기관'] = finance['금융기관'].apply(lambda x: int(x))
finance['금융기관'].unique()

array([21,  3,  1,  2,  4, 41,  7,  0,  5, 32, 18, 53,  9, 15, 10, 17, 11,
        6,  8, 25, 61, 13, 22, 29, 14, 12, 24, 50])

In [8]:
finance = finance[['구동', '금융기관']]
finance.head()

,구동,금융기관
0,종로구 사직동,21
1,종로구 삼청동,3
2,종로구 부암동,1
3,종로구 평창동,2
4,종로구 무악동,1


In [9]:
finance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426 entries, 0 to 425
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   구동      426 non-null    object
 1   금융기관    426 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 6.8+ KB


In [10]:
finance.head()

,구동,금융기관
0,종로구 사직동,21
1,종로구 삼청동,3
2,종로구 부암동,1
3,종로구 평창동,2
4,종로구 무악동,1


## 유통업체현황

In [11]:
mart = pd.read_csv(f'{drive_path}data/경제/유통업체현황_20240411161623.csv', header= 1)
mart.head()

,동별(1),동별(2),동별(3),합계,대형마트,백화점,전문점,쇼핑센터,복합쇼핑몰,그밖의 대규모점포
0,동별(1),동별(2),동별(3),소계,소계,소계,소계,소계,소계,소계
1,동별(1),동별(2),동별(3),개소 (개소),개소 (개소),개소 (개소),개소 (개소),개소 (개소),개소 (개소),개소 (개소)
2,서울시,소계,소계,444,55,31,24,41,8,285
3,서울시,종로구,소계,13,-,-,2,-,-,11
4,서울시,종로구,사직동,-,-,-,-,-,-,-


In [12]:
mart = mart[(mart['동별(3)'] != '동별(3)') & (mart['동별(3)'] != '소계')]
mart.head()

,동별(1),동별(2),동별(3),합계,대형마트,백화점,전문점,쇼핑센터,복합쇼핑몰,그밖의 대규모점포
4,서울시,종로구,사직동,-,-,-,-,-,-,-
5,서울시,종로구,삼청동,-,-,-,-,-,-,-
6,서울시,종로구,부암동,-,-,-,-,-,-,-
7,서울시,종로구,평창동,-,-,-,-,-,-,-
8,서울시,종로구,무악동,-,-,-,-,-,-,-


In [13]:
mart.drop(columns = '동별(1)', inplace =True)
selected_colunms =  ['시군구', '읍면동', '유통업체합계'] +mart.columns[3:].tolist()
mart.columns = selected_colunms
mart.reset_index(drop = True, inplace = True)
mart.head()

,시군구,읍면동,유통업체합계,대형마트,백화점,전문점,쇼핑센터,복합쇼핑몰,그밖의 대규모점포
0,종로구,사직동,-,-,-,-,-,-,-
1,종로구,삼청동,-,-,-,-,-,-,-
2,종로구,부암동,-,-,-,-,-,-,-
3,종로구,평창동,-,-,-,-,-,-,-
4,종로구,무악동,-,-,-,-,-,-,-


In [14]:
mart['읍면동'] = mart['읍면동'].apply(lambda x: re.sub(r'\·', '', x))
mart['읍면동'] = mart['읍면동'].apply(lambda x: re.sub(r'\.', '', x))
mart[mart['읍면동'].str.contains('\.') | mart['읍면동'].str.contains('\·') ]

,시군구,읍면동,유통업체합계,대형마트,백화점,전문점,쇼핑센터,복합쇼핑몰,그밖의 대규모점포


In [15]:
mart['구동'] = mart['시군구'] + ' ' +mart['읍면동']
mart.head()

,시군구,읍면동,유통업체합계,대형마트,백화점,전문점,쇼핑센터,복합쇼핑몰,그밖의 대규모점포,구동
0,종로구,사직동,-,-,-,-,-,-,-,종로구 사직동
1,종로구,삼청동,-,-,-,-,-,-,-,종로구 삼청동
2,종로구,부암동,-,-,-,-,-,-,-,종로구 부암동
3,종로구,평창동,-,-,-,-,-,-,-,종로구 평창동
4,종로구,무악동,-,-,-,-,-,-,-,종로구 무악동


In [16]:
mart = mart.replace('-', 0)
mart['유통업체합계'].unique()

array([0, '7', '3', '1', '9', '6', '12', '16', '2', '5', '4'],
      dtype=object)

In [17]:
mart.drop(columns = ['시군구', '읍면동'], inplace = True)

In [18]:
mart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426 entries, 0 to 425
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   유통업체합계     426 non-null    object
 1   대형마트       426 non-null    object
 2   백화점        426 non-null    object
 3   전문점        426 non-null    object
 4   쇼핑센터       426 non-null    object
 5   복합쇼핑몰      426 non-null    object
 6   그밖의 대규모점포  426 non-null    object
 7   구동         426 non-null    object
dtypes: object(8)
memory usage: 26.8+ KB


In [19]:
mart.head(5)

,유통업체합계,대형마트,백화점,전문점,쇼핑센터,복합쇼핑몰,그밖의 대규모점포,구동
0,0,0,0,0,0,0,0,종로구 사직동
1,0,0,0,0,0,0,0,종로구 삼청동
2,0,0,0,0,0,0,0,종로구 부암동
3,0,0,0,0,0,0,0,종로구 평창동
4,0,0,0,0,0,0,0,종로구 무악동


## 사업체 현황

In [20]:
company = pd.read_csv(f'{drive_path}data/경제/사업체현황(산업대분류별_동별)_20240411161747.csv', header = 2)
company.head()

,동별(1),동별(2),동별(3),소계,농업 임업 및 어업,광업,제조업,전기 가스 증기 및 공기조절 공급업,수도 하수 및 폐기물 처리 원료 재생업,건설업,...,정보통신업,금융 및 보험업,부동산업,전문 과학 및 기술 서비스업,사업시설 관리 사업 지원 및 임대 서비스업,공공행정 국방 및 사회보장 행정,교육 서비스업,보건업 및 사회복지 서비스업,예술 스포츠 및 여가관련 서비스업,협회 및 단체 수리 및 기타 개인 서비스업
0,동별(1),동별(2),동별(3),사업체수 (개),사업체수 (개),사업체수 (개),사업체수 (개),사업체수 (개),사업체수 (개),사업체수 (개),...,사업체수 (개),사업체수 (개),사업체수 (개),사업체수 (개),사업체수 (개),사업체수 (개),사업체수 (개),사업체수 (개),사업체수 (개),사업체수 (개)
1,합계,소계,소계,1180025,141,26,75641,653,764,66868,...,59783,24693,72058,81166,29595,1299,47002,32319,28242,77916
2,합계,종로구,소계,47595,4,4,5312,57,8,594,...,1565,942,2571,2692,1463,93,1067,841,1118,2606
3,합계,종로구,사직동,4994,1,1,172,10,1,59,...,358,289,319,634,435,28,122,125,115,288
4,합계,종로구,삼청동,970,-,-,38,-,-,13,...,40,13,63,92,29,3,22,12,30,45


In [21]:
company = company[(company['동별(3)'] != '동별(3)') & (company['동별(3)'] != '소계')]
company.head()

,동별(1),동별(2),동별(3),소계,농업 임업 및 어업,광업,제조업,전기 가스 증기 및 공기조절 공급업,수도 하수 및 폐기물 처리 원료 재생업,건설업,...,정보통신업,금융 및 보험업,부동산업,전문 과학 및 기술 서비스업,사업시설 관리 사업 지원 및 임대 서비스업,공공행정 국방 및 사회보장 행정,교육 서비스업,보건업 및 사회복지 서비스업,예술 스포츠 및 여가관련 서비스업,협회 및 단체 수리 및 기타 개인 서비스업
3,합계,종로구,사직동,4994,1,1,172,10,1,59,...,358,289,319,634,435,28,122,125,115,288
4,합계,종로구,삼청동,970,-,-,38,-,-,13,...,40,13,63,92,29,3,22,12,30,45
5,합계,종로구,부암동,908,-,-,44,-,-,33,...,68,2,51,93,20,3,54,29,39,78
6,합계,종로구,평창동,1286,-,1,39,-,1,42,...,98,11,109,115,30,3,64,42,74,121
7,합계,종로구,무악동,837,-,-,6,-,-,15,...,21,4,28,26,13,1,28,12,13,49


In [22]:
company.drop(columns = '동별(1)', inplace =True)
selected_colunms =  ['시군구', '읍면동', '사업체합계'] +company.columns[3:].tolist()
company.columns = selected_colunms
company.reset_index(drop = True, inplace = True)
company.head()

,시군구,읍면동,사업체합계,농업 임업 및 어업,광업,제조업,전기 가스 증기 및 공기조절 공급업,수도 하수 및 폐기물 처리 원료 재생업,건설업,도매 및 소매업,...,정보통신업,금융 및 보험업,부동산업,전문 과학 및 기술 서비스업,사업시설 관리 사업 지원 및 임대 서비스업,공공행정 국방 및 사회보장 행정,교육 서비스업,보건업 및 사회복지 서비스업,예술 스포츠 및 여가관련 서비스업,협회 및 단체 수리 및 기타 개인 서비스업
0,종로구,사직동,4994,1,1,172,10,1,59,781,...,358,289,319,634,435,28,122,125,115,288
1,종로구,삼청동,970,-,-,38,-,-,13,318,...,40,13,63,92,29,3,22,12,30,45
2,종로구,부암동,908,-,-,44,-,-,33,235,...,68,2,51,93,20,3,54,29,39,78
3,종로구,평창동,1286,-,1,39,-,1,42,347,...,98,11,109,115,30,3,64,42,74,121
4,종로구,무악동,837,-,-,6,-,-,15,98,...,21,4,28,26,13,1,28,12,13,49


In [23]:
company['읍면동'] = company['읍면동'].apply(lambda x: re.sub(r'\·', '', x))
company['읍면동'] = company['읍면동'].apply(lambda x: re.sub(r'\.', '', x))
company[company['읍면동'].str.contains('\.') | company['읍면동'].str.contains('\·') ]

,시군구,읍면동,사업체합계,농업 임업 및 어업,광업,제조업,전기 가스 증기 및 공기조절 공급업,수도 하수 및 폐기물 처리 원료 재생업,건설업,도매 및 소매업,...,정보통신업,금융 및 보험업,부동산업,전문 과학 및 기술 서비스업,사업시설 관리 사업 지원 및 임대 서비스업,공공행정 국방 및 사회보장 행정,교육 서비스업,보건업 및 사회복지 서비스업,예술 스포츠 및 여가관련 서비스업,협회 및 단체 수리 및 기타 개인 서비스업


In [24]:
company['구동'] = company['시군구'] + ' ' +company['읍면동']
company.head()

,시군구,읍면동,사업체합계,농업 임업 및 어업,광업,제조업,전기 가스 증기 및 공기조절 공급업,수도 하수 및 폐기물 처리 원료 재생업,건설업,도매 및 소매업,...,금융 및 보험업,부동산업,전문 과학 및 기술 서비스업,사업시설 관리 사업 지원 및 임대 서비스업,공공행정 국방 및 사회보장 행정,교육 서비스업,보건업 및 사회복지 서비스업,예술 스포츠 및 여가관련 서비스업,협회 및 단체 수리 및 기타 개인 서비스업,구동
0,종로구,사직동,4994,1,1,172,10,1,59,781,...,289,319,634,435,28,122,125,115,288,종로구 사직동
1,종로구,삼청동,970,-,-,38,-,-,13,318,...,13,63,92,29,3,22,12,30,45,종로구 삼청동
2,종로구,부암동,908,-,-,44,-,-,33,235,...,2,51,93,20,3,54,29,39,78,종로구 부암동
3,종로구,평창동,1286,-,1,39,-,1,42,347,...,11,109,115,30,3,64,42,74,121,종로구 평창동
4,종로구,무악동,837,-,-,6,-,-,15,98,...,4,28,26,13,1,28,12,13,49,종로구 무악동


In [25]:
# company이 없는 것은 - 로 값이 입력되어있다.
# 이를 0으로 변환
company = company.replace('-', 0)


In [26]:
company.drop(columns = ['시군구', '읍면동'], inplace = True)
company.head()

,사업체합계,농업 임업 및 어업,광업,제조업,전기 가스 증기 및 공기조절 공급업,수도 하수 및 폐기물 처리 원료 재생업,건설업,도매 및 소매업,운수 및 창고업,숙박 및 음식점업,...,금융 및 보험업,부동산업,전문 과학 및 기술 서비스업,사업시설 관리 사업 지원 및 임대 서비스업,공공행정 국방 및 사회보장 행정,교육 서비스업,보건업 및 사회복지 서비스업,예술 스포츠 및 여가관련 서비스업,협회 및 단체 수리 및 기타 개인 서비스업,구동
0,4994,1,1,172,10,1,59,781,324,932,...,289,319,634,435,28,122,125,115,288,종로구 사직동
1,970,0,0,38,0,0,13,318,19,233,...,13,63,92,29,3,22,12,30,45,종로구 삼청동
2,908,0,0,44,0,0,33,235,33,126,...,2,51,93,20,3,54,29,39,78,종로구 부암동
3,1286,0,1,39,0,1,42,347,64,125,...,11,109,115,30,3,64,42,74,121,종로구 평창동
4,837,0,0,6,0,0,15,98,483,40,...,4,28,26,13,1,28,12,13,49,종로구 무악동


In [27]:
company.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426 entries, 0 to 425
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   사업체합계                    426 non-null    object
 1   농업 임업 및 어업               426 non-null    object
 2   광업                       426 non-null    object
 3   제조업                      426 non-null    object
 4   전기 가스 증기 및 공기조절 공급업      426 non-null    object
 5   수도 하수 및 폐기물 처리 원료 재생업    426 non-null    object
 6   건설업                      426 non-null    object
 7   도매 및 소매업                 426 non-null    object
 8   운수 및 창고업                 426 non-null    object
 9   숙박 및 음식점업                426 non-null    object
 10  정보통신업                    426 non-null    object
 11  금융 및 보험업                 426 non-null    object
 12  부동산업                     426 non-null    object
 13  전문 과학 및 기술 서비스업          426 non-null    object
 14  사업시설 관리 사업 지원 및 임대 서비스업  426 non-null    o

## 종사자

In [28]:
labor = pd.read_csv(f'{drive_path}data/경제/사업체+및+종사자+밀도(동별)_20240411161837.csv', header = 1)
labor.head()

,동별(1),동별(2),동별(3),사업체수 (개),종사자수 (명),평균종사자 (명),사업체밀도 (null),종사자밀도(A) (명/㎢),인구밀도(B),종사자밀도비(A/B*100)
0,합계,소계,소계,1180025,5795425,4.91,1949.78,9575.92,15974.12,59.95
1,합계,종로구,소계,47595,278403,5.85,1990.33,11642.25,6365.16,182.91
2,합계,종로구,사직동,4994,56046,11.22,4060.16,45565.85,7605.69,599.10
3,합계,종로구,삼청동,970,5118,5.28,651.01,3434.90,1773.15,193.72
4,합계,종로구,부암동,908,4140,4.56,400.00,1823.79,4200.88,43.41


In [29]:
labor = labor[(labor['동별(3)'] != '동별(3)') & (labor['동별(3)'] != '소계')]
labor.head()

,동별(1),동별(2),동별(3),사업체수 (개),종사자수 (명),평균종사자 (명),사업체밀도 (null),종사자밀도(A) (명/㎢),인구밀도(B),종사자밀도비(A/B*100)
2,합계,종로구,사직동,4994,56046,11.22,4060.16,45565.85,7605.69,599.10
3,합계,종로구,삼청동,970,5118,5.28,651.01,3434.90,1773.15,193.72
4,합계,종로구,부암동,908,4140,4.56,400.00,1823.79,4200.88,43.41
5,합계,종로구,평창동,1286,4294,3.34,144.98,484.10,2013.30,24.05
6,합계,종로구,무악동,837,1987,2.37,2325.00,5519.44,22366.67,24.68


In [30]:
labor.drop(columns = '동별(1)', inplace =True)
selected_colunms =  ['시군구', '읍면동', '사업체합계_종사자통계', '종사자수', '평균종사자', '사업체밀도', '종사자밀도', '인구밀도', '종사자밀도비']
labor.columns = selected_colunms
labor.reset_index(drop = True, inplace = True)
labor.head()

,시군구,읍면동,사업체합계_종사자통계,종사자수,평균종사자,사업체밀도,종사자밀도,인구밀도,종사자밀도비
0,종로구,사직동,4994,56046,11.22,4060.16,45565.85,7605.69,599.10
1,종로구,삼청동,970,5118,5.28,651.01,3434.90,1773.15,193.72
2,종로구,부암동,908,4140,4.56,400.00,1823.79,4200.88,43.41
3,종로구,평창동,1286,4294,3.34,144.98,484.10,2013.30,24.05
4,종로구,무악동,837,1987,2.37,2325.00,5519.44,22366.67,24.68


In [31]:
labor['읍면동'] = labor['읍면동'].apply(lambda x: re.sub(r'\·', '', x))
labor['읍면동'] = labor['읍면동'].apply(lambda x: re.sub(r'\.', '', x))
labor[labor['읍면동'].str.contains('\.') | labor['읍면동'].str.contains('\·') ]

,시군구,읍면동,사업체합계_종사자통계,종사자수,평균종사자,사업체밀도,종사자밀도,인구밀도,종사자밀도비


In [32]:
labor['구동'] = labor['시군구'] + ' ' +labor['읍면동']
labor.head()

,시군구,읍면동,사업체합계_종사자통계,종사자수,평균종사자,사업체밀도,종사자밀도,인구밀도,종사자밀도비,구동
0,종로구,사직동,4994,56046,11.22,4060.16,45565.85,7605.69,599.10,종로구 사직동
1,종로구,삼청동,970,5118,5.28,651.01,3434.90,1773.15,193.72,종로구 삼청동
2,종로구,부암동,908,4140,4.56,400.00,1823.79,4200.88,43.41,종로구 부암동
3,종로구,평창동,1286,4294,3.34,144.98,484.10,2013.30,24.05,종로구 평창동
4,종로구,무악동,837,1987,2.37,2325.00,5519.44,22366.67,24.68,종로구 무악동


In [33]:
labor = labor.replace('-', 0)

In [34]:
labor.drop(columns = ['시군구', '읍면동'], inplace = True)
labor.head()

,사업체합계_종사자통계,종사자수,평균종사자,사업체밀도,종사자밀도,인구밀도,종사자밀도비,구동
0,4994,56046,11.22,4060.16,45565.85,7605.69,599.10,종로구 사직동
1,970,5118,5.28,651.01,3434.90,1773.15,193.72,종로구 삼청동
2,908,4140,4.56,400.00,1823.79,4200.88,43.41,종로구 부암동
3,1286,4294,3.34,144.98,484.10,2013.30,24.05,종로구 평창동
4,837,1987,2.37,2325.00,5519.44,22366.67,24.68,종로구 무악동


In [35]:
labor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426 entries, 0 to 425
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   사업체합계_종사자통계  426 non-null    int64  
 1   종사자수         426 non-null    int64  
 2   평균종사자        426 non-null    float64
 3   사업체밀도        426 non-null    float64
 4   종사자밀도        426 non-null    float64
 5   인구밀도         426 non-null    float64
 6   종사자밀도비       426 non-null    float64
 7   구동           426 non-null    object 
dtypes: float64(5), int64(2), object(1)
memory usage: 26.8+ KB


# 지도 파일과 결합

In [36]:
def merge_map(df_name, key):

    region_code = pd.read_csv(f'{drive_path}final/data/region_code.csv')
    dong_shp = gpd.read_file(f'{drive_path}data/seoul_shp/dong/bnd_dong_11_2023_2023_2Q.shp')

    # 코드명 타입 일치
    region_code['ADM_CD'] = region_code['ADM_CD'].astype('int')
    dong_shp['ADM_CD'] = dong_shp['ADM_CD'].astype('int')

    # 지도파일과 지역코드 결합
    map_base = dong_shp.merge(region_code,
                          how = 'left',
                          on = 'ADM_CD')
    map_base['ADM_NM'] = map_base['ADM_NM'].apply(lambda x: re.sub(r'\·', '', x))

    # 동이름 다른것에 대응하기 위해서
    map_base['key1'] = map_base['시군구'] + ' ' + map_base['ADM_NM']
    map_base['key2'] = map_base['시군구'] + ' ' + map_base['읍면동']

    if '구동' in df_name.columns:
        pass
    else :
        df_name['구동'] = df_name['자치구'] + ' ' + df_name['행정동명']

    globals()['final'] = map_base.merge(df_name.rename(columns = {'구동' : key}),
                                        how = 'left',
                                        on = key)

    final.dropna(inplace = True)

In [37]:
merge_map(finance, 'key1')
final.head()

,BASE_DATE,ADM_NM,ADM_CD,geometry,dong_name,통계청코드,도로명코드10,통계청코드8,도로명코드8,도로명코드,시도명,시군구,읍면동,key1,key2,금융기관
0,20230701,사직동,11010530,"POLYGON ((953553.932 1953335.741, 953555.211 1...",서울특별시 종로구 사직동,1101053.0,1.111053e+09,11010530.0,11110530.0,11110530.0,서울특별시,종로구,사직동,종로구 사직동,종로구 사직동,21.0
1,20230701,삼청동,11010540,"POLYGON ((954025.242 1953916.389, 954026.972 1...",서울특별시 종로구 삼청동,1101054.0,1.111054e+09,11010540.0,11110540.0,11110540.0,서울특별시,종로구,삼청동,종로구 삼청동,종로구 삼청동,3.0
2,20230701,부암동,11010550,"POLYGON ((952490.380 1956548.821, 952497.594 1...",서울특별시 종로구 부암동,1101055.0,1.111055e+09,11010550.0,11110550.0,11110550.0,서울특별시,종로구,부암동,종로구 부암동,종로구 부암동,1.0
3,20230701,평창동,11010560,"POLYGON ((953683.828 1959209.871, 953665.283 1...",서울특별시 종로구 평창동,1101056.0,1.111056e+09,11010560.0,11110560.0,11110560.0,서울특별시,종로구,평창동,종로구 평창동,종로구 평창동,2.0
4,20230701,한남동,11030740,"POLYGON ((956238.296 1950166.610, 956237.942 1...",서울특별시 용산구 한남동,1103074.0,1.117068e+09,11030740.0,11170685.0,11170685.0,서울특별시,용산구,한남동,용산구 한남동,용산구 한남동,11.0


In [38]:
final[final.isna().any(axis=1)]

,BASE_DATE,ADM_NM,ADM_CD,geometry,dong_name,통계청코드,도로명코드10,통계청코드8,도로명코드8,도로명코드,시도명,시군구,읍면동,key1,key2,금융기관


In [39]:
# mart
final = final.merge(mart.rename(columns = {'구동' : 'key1'}),
                       how = 'left',
                       on = 'key1')
final.head()

,BASE_DATE,ADM_NM,ADM_CD,geometry,dong_name,통계청코드,도로명코드10,통계청코드8,도로명코드8,도로명코드,...,key1,key2,금융기관,유통업체합계,대형마트,백화점,전문점,쇼핑센터,복합쇼핑몰,그밖의 대규모점포
0,20230701,사직동,11010530,"POLYGON ((953553.932 1953335.741, 953555.211 1...",서울특별시 종로구 사직동,1101053.0,1.111053e+09,11010530.0,11110530.0,11110530.0,...,종로구 사직동,종로구 사직동,21.0,0,0,0,0,0,0,0
1,20230701,삼청동,11010540,"POLYGON ((954025.242 1953916.389, 954026.972 1...",서울특별시 종로구 삼청동,1101054.0,1.111054e+09,11010540.0,11110540.0,11110540.0,...,종로구 삼청동,종로구 삼청동,3.0,0,0,0,0,0,0,0
2,20230701,부암동,11010550,"POLYGON ((952490.380 1956548.821, 952497.594 1...",서울특별시 종로구 부암동,1101055.0,1.111055e+09,11010550.0,11110550.0,11110550.0,...,종로구 부암동,종로구 부암동,1.0,0,0,0,0,0,0,0
3,20230701,평창동,11010560,"POLYGON ((953683.828 1959209.871, 953665.283 1...",서울특별시 종로구 평창동,1101056.0,1.111056e+09,11010560.0,11110560.0,11110560.0,...,종로구 평창동,종로구 평창동,2.0,0,0,0,0,0,0,0
4,20230701,한남동,11030740,"POLYGON ((956238.296 1950166.610, 956237.942 1...",서울특별시 용산구 한남동,1103074.0,1.117068e+09,11030740.0,11170685.0,11170685.0,...,용산구 한남동,용산구 한남동,11.0,0,0,0,0,0,0,0


In [40]:
final = final.merge(company.rename(columns = {'구동' : 'key1'}),
                       how = 'left',
                       on = 'key1')
final.head()

,BASE_DATE,ADM_NM,ADM_CD,geometry,dong_name,통계청코드,도로명코드10,통계청코드8,도로명코드8,도로명코드,...,정보통신업,금융 및 보험업,부동산업,전문 과학 및 기술 서비스업,사업시설 관리 사업 지원 및 임대 서비스업,공공행정 국방 및 사회보장 행정,교육 서비스업,보건업 및 사회복지 서비스업,예술 스포츠 및 여가관련 서비스업,협회 및 단체 수리 및 기타 개인 서비스업
0,20230701,사직동,11010530,"POLYGON ((953553.932 1953335.741, 953555.211 1...",서울특별시 종로구 사직동,1101053.0,1.111053e+09,11010530.0,11110530.0,11110530.0,...,358,289,319,634,435,28,122,125,115,288
1,20230701,삼청동,11010540,"POLYGON ((954025.242 1953916.389, 954026.972 1...",서울특별시 종로구 삼청동,1101054.0,1.111054e+09,11010540.0,11110540.0,11110540.0,...,40,13,63,92,29,3,22,12,30,45
2,20230701,부암동,11010550,"POLYGON ((952490.380 1956548.821, 952497.594 1...",서울특별시 종로구 부암동,1101055.0,1.111055e+09,11010550.0,11110550.0,11110550.0,...,68,2,51,93,20,3,54,29,39,78
3,20230701,평창동,11010560,"POLYGON ((953683.828 1959209.871, 953665.283 1...",서울특별시 종로구 평창동,1101056.0,1.111056e+09,11010560.0,11110560.0,11110560.0,...,98,11,109,115,30,3,64,42,74,121
4,20230701,한남동,11030740,"POLYGON ((956238.296 1950166.610, 956237.942 1...",서울특별시 용산구 한남동,1103074.0,1.117068e+09,11030740.0,11170685.0,11170685.0,...,174,82,317,348,84,3,92,51,116,208


In [41]:
final = final.merge(labor.rename(columns = {'구동' : 'key1'}),
                       how = 'left',
                       on = 'key1')
final.head()

,BASE_DATE,ADM_NM,ADM_CD,geometry,dong_name,통계청코드,도로명코드10,통계청코드8,도로명코드8,도로명코드,...,보건업 및 사회복지 서비스업,예술 스포츠 및 여가관련 서비스업,협회 및 단체 수리 및 기타 개인 서비스업,사업체합계_종사자통계,종사자수,평균종사자,사업체밀도,종사자밀도,인구밀도,종사자밀도비
0,20230701,사직동,11010530,"POLYGON ((953553.932 1953335.741, 953555.211 1...",서울특별시 종로구 사직동,1101053.0,1.111053e+09,11010530.0,11110530.0,11110530.0,...,125,115,288,4994,56046,11.22,4060.16,45565.85,7605.69,599.10
1,20230701,삼청동,11010540,"POLYGON ((954025.242 1953916.389, 954026.972 1...",서울특별시 종로구 삼청동,1101054.0,1.111054e+09,11010540.0,11110540.0,11110540.0,...,12,30,45,970,5118,5.28,651.01,3434.90,1773.15,193.72
2,20230701,부암동,11010550,"POLYGON ((952490.380 1956548.821, 952497.594 1...",서울특별시 종로구 부암동,1101055.0,1.111055e+09,11010550.0,11110550.0,11110550.0,...,29,39,78,908,4140,4.56,400.00,1823.79,4200.88,43.41
3,20230701,평창동,11010560,"POLYGON ((953683.828 1959209.871, 953665.283 1...",서울특별시 종로구 평창동,1101056.0,1.111056e+09,11010560.0,11110560.0,11110560.0,...,42,74,121,1286,4294,3.34,144.98,484.10,2013.30,24.05
4,20230701,한남동,11030740,"POLYGON ((956238.296 1950166.610, 956237.942 1...",서울특별시 용산구 한남동,1103074.0,1.117068e+09,11030740.0,11170685.0,11170685.0,...,51,116,208,3213,20440,6.36,1067.44,6790.70,7326.91,92.68


In [42]:
final.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 421 entries, 0 to 420
Data columns (total 50 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   BASE_DATE                421 non-null    object  
 1   ADM_NM                   421 non-null    object  
 2   ADM_CD                   421 non-null    int64   
 3   geometry                 421 non-null    geometry
 4   dong_name                421 non-null    object  
 5   통계청코드                    421 non-null    float64 
 6   도로명코드10                  421 non-null    float64 
 7   통계청코드8                   421 non-null    float64 
 8   도로명코드8                   421 non-null    float64 
 9   도로명코드                    421 non-null    float64 
 10  시도명                      421 non-null    object  
 11  시군구                      421 non-null    object  
 12  읍면동                      421 non-null    object  
 13  key1                     421 non-null    object  
 14  ke

In [43]:
convert_columns = final.columns[15:]

final[convert_columns] = final[convert_columns].astype(float)

In [44]:
final.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 421 entries, 0 to 420
Data columns (total 50 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   BASE_DATE                421 non-null    object  
 1   ADM_NM                   421 non-null    object  
 2   ADM_CD                   421 non-null    int64   
 3   geometry                 421 non-null    geometry
 4   dong_name                421 non-null    object  
 5   통계청코드                    421 non-null    float64 
 6   도로명코드10                  421 non-null    float64 
 7   통계청코드8                   421 non-null    float64 
 8   도로명코드8                   421 non-null    float64 
 9   도로명코드                    421 non-null    float64 
 10  시도명                      421 non-null    object  
 11  시군구                      421 non-null    object  
 12  읍면동                      421 non-null    object  
 13  key1                     421 non-null    object  
 14  ke

In [45]:
final.head()

,BASE_DATE,ADM_NM,ADM_CD,geometry,dong_name,통계청코드,도로명코드10,통계청코드8,도로명코드8,도로명코드,...,보건업 및 사회복지 서비스업,예술 스포츠 및 여가관련 서비스업,협회 및 단체 수리 및 기타 개인 서비스업,사업체합계_종사자통계,종사자수,평균종사자,사업체밀도,종사자밀도,인구밀도,종사자밀도비
0,20230701,사직동,11010530,"POLYGON ((953553.932 1953335.741, 953555.211 1...",서울특별시 종로구 사직동,1101053.0,1.111053e+09,11010530.0,11110530.0,11110530.0,...,125.0,115.0,288.0,4994.0,56046.0,11.22,4060.16,45565.85,7605.69,599.10
1,20230701,삼청동,11010540,"POLYGON ((954025.242 1953916.389, 954026.972 1...",서울특별시 종로구 삼청동,1101054.0,1.111054e+09,11010540.0,11110540.0,11110540.0,...,12.0,30.0,45.0,970.0,5118.0,5.28,651.01,3434.90,1773.15,193.72
2,20230701,부암동,11010550,"POLYGON ((952490.380 1956548.821, 952497.594 1...",서울특별시 종로구 부암동,1101055.0,1.111055e+09,11010550.0,11110550.0,11110550.0,...,29.0,39.0,78.0,908.0,4140.0,4.56,400.00,1823.79,4200.88,43.41
3,20230701,평창동,11010560,"POLYGON ((953683.828 1959209.871, 953665.283 1...",서울특별시 종로구 평창동,1101056.0,1.111056e+09,11010560.0,11110560.0,11110560.0,...,42.0,74.0,121.0,1286.0,4294.0,3.34,144.98,484.10,2013.30,24.05
4,20230701,한남동,11030740,"POLYGON ((956238.296 1950166.610, 956237.942 1...",서울특별시 용산구 한남동,1103074.0,1.117068e+09,11030740.0,11170685.0,11170685.0,...,51.0,116.0,208.0,3213.0,20440.0,6.36,1067.44,6790.70,7326.91,92.68


In [46]:
final[final.isna().any(axis=1)]

,BASE_DATE,ADM_NM,ADM_CD,geometry,dong_name,통계청코드,도로명코드10,통계청코드8,도로명코드8,도로명코드,...,보건업 및 사회복지 서비스업,예술 스포츠 및 여가관련 서비스업,협회 및 단체 수리 및 기타 개인 서비스업,사업체합계_종사자통계,종사자수,평균종사자,사업체밀도,종사자밀도,인구밀도,종사자밀도비


In [47]:
final.to_csv(f'{drive_path}final/data/song1.csv', index = False)